In [1]:
import pandas as pd

#загрузим реестр преподавателей от оператора
teacher_xls =\
pd.ExcelFile('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/Лабы,_МК_факт_мэтч_с_хардами,_Я_диск_и_наоборот,_вместе_с_потеряшками_fixed.xlsx')
teachers = pd.read_excel(teacher_xls, 'Уникальные препода').fillna(0)
workshops = pd.read_excel(teacher_xls, 'Мастер-классы').fillna(0)
laboratory = pd.read_excel(teacher_xls, 'Лаборатории').fillna(0)

teachers['Имя']=teachers.I1
teachers['Фамилия']=teachers.apply(lambda x: x.I2 if x.I3==0 else x.I3, axis=1)

#Загрузим список преподов с платформы
platform_xls = pd.ExcelFile('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/Преподы_и_мероприяти_от_Тагира_с_треком_и_ФИО.xlsx')
platform = pd.read_excel(platform_xls, 'Уникальные Преподы')

platform = platform.rename(columns={ 'name':'Имя Фамилия','ccc': 'Имя',
                                 'bbb':'Фамилия'})
#Сошьем данные о преподах
teachers_platform = pd.merge(
    teachers,
    platform,
    on=['Имя','Фамилия'],
    how='left' ).fillna(0).reset_index()

teachers_platform['Имя']=  teachers_platform['Имя'].astype('str')
teachers_platform['Фамилия']=  teachers_platform['Фамилия'].astype('str')
teachers_platform['leaderID']=  teachers_platform['leaderID'].astype('int')
teachers_platform['Имя Фамилия']=  teachers_platform['Имя Фамилия'].astype('str')

tp_grouped = teachers_platform[['Преподаватель','Имя Фамилия','leaderID']]
tp_grouped = tp_grouped.rename(columns={'Имя Фамилия':'Имя на платформе'})

tp_grouped['Преподаватель'] = tp_grouped['Преподаватель'].apply(lambda x: x.lstrip())
tp_grouped['Преподаватель'] = tp_grouped['Преподаватель'].apply(lambda x: x.rstrip())

In [2]:
#Транспонируем данные о лабораториях и мастер-классах
teachers_list = ['Преподаватель1', 'Преподаватель2', 'Преподаватель3', 'Преподаватель4', 'Преподаватель5']

columns=['Название МЕРОПРИЯТИЯ', 'Название+дата', 'Формат мероприятия', 'ТРЭК',
         'Дата', 'Время', 'Длительность','Тип кратко','Преподаватель']

workshop_list =  list()

for x in teachers_list:
    elem =  workshops[workshops[x]!=0][['Название МЕРОПРИЯТИЯ', 'Название+дата', 'Формат мероприятия', 'ТРЭК',
                                        'Дата', 'Время', 'Длительность','Тип кратко',x]]
    elem = elem.rename(columns={x:'Преподаватель'})
    workshop_list.append(elem)

workshops_concat=pd.concat([workshop_list[1],workshop_list[2],workshop_list[3],
                                  workshop_list[4],workshop_list[0]]).fillna(0)
#Обрежем пробелы слева и справа
workshops_concat['Преподаватель'] = workshops_concat['Преподаватель'].astype('str')
workshops_concat['Преподаватель'] = workshops_concat['Преподаватель'].apply(lambda x: x.lstrip())
workshops_concat['Преподаватель'] = workshops_concat['Преподаватель'].apply(lambda x: x.rstrip())

teachers_list = ['Преподаватель 1', 'Преподаватель 2','Преподаватель 3', 'Преподаватель 4', 'Преподаватель 5',
                 'Преподаватель 6', 'Преподаватель 7', 'Преподаватель 8','Преподаватель 9', 'Преподаватель 10', 
                 'Преподаватель 11','Преподаватель 12', 'Преподаватель 13', 'Преподаватель 14','Преподаватель 15', 
                 'Преподаватель 16', 'Преподаватель 17','Преподаватель 18', 'Преподаватель 19']

laboratory_list =  list()

for x in teachers_list:
    elem =  laboratory[laboratory[x]!=0][[ 'НАЗВАНИЕ МЕРОПРИЯТИЯ', 'Описание + дата', 'Формат мероприятия',
                                           'ТРЭК', 'Дата', 'Длительность','Тип кратко',x]]
    elem = elem.rename(columns={x:'Преподаватель'})
    laboratory_list.append(elem)
    
laboratory_concat=pd.concat([laboratory_list[0], laboratory_list[1], laboratory_list[2], laboratory_list[3], laboratory_list[4],
                                 laboratory_list[5], laboratory_list[6], laboratory_list[7], laboratory_list[8], laboratory_list[9],
                                 laboratory_list[10],laboratory_list[11],laboratory_list[12],laboratory_list[13],laboratory_list[14],
                                 laboratory_list[15],laboratory_list[16],laboratory_list[17],laboratory_list[18]]).fillna(0)

#Обрежем пробелы слева и справа
laboratory_concat['Преподаватель'] = laboratory_concat['Преподаватель'].astype('str')
laboratory_concat['Преподаватель'] = laboratory_concat['Преподаватель'].apply(lambda x: x.lstrip())
laboratory_concat['Преподаватель'] = laboratory_concat['Преподаватель'].apply(lambda x: x.rstrip())
#Теперь сошьем наши длинные данные с именами преподов - так чтобы присвоить имена из БД и leaderID

workshops_by_teacher = pd.merge(
    workshops_concat,
    tp_grouped,
    on=['Преподаватель'],
    how='left' ).fillna(0).reset_index()
workshops_by_teacher.leaderID=workshops_by_teacher.leaderID.astype('int')

laboratory_by_teacher = pd.merge(
    laboratory_concat,
    tp_grouped,
    on=['Преподаватель'],
    how='left' ).fillna(0).reset_index()
laboratory_by_teacher.leaderID=laboratory_by_teacher.leaderID.astype('int')

#Приведем колонки к одному названию
laboratory_by_teacher = laboratory_by_teacher.rename(columns={'НАЗВАНИЕ МЕРОПРИЯТИЯ':'Мероприятие',
                                                              'Описание + дата':'Название+дата'})
workshops_by_teacher = workshops_by_teacher.rename(columns={'Название МЕРОПРИЯТИЯ':'Мероприятие'})

#Теперь склеим их
events_by_teacher=pd.concat(
    [laboratory_by_teacher[['Преподаватель','Имя на платформе','leaderID','Мероприятие','Дата','Формат мероприятия','ТРЭК','Длительность']],
     workshops_by_teacher[['Преподаватель','Имя на платформе','leaderID','Мероприятие','Дата','Формат мероприятия','ТРЭК','Длительность']]]).\
fillna(0)

events_by_teacher['Имя на платформе']=events_by_teacher['Имя на платформе'].astype('str')
events_by_teacher['ТРЭК']=events_by_teacher['ТРЭК'].astype('str')
events_by_teacher.leaderID=events_by_teacher.leaderID.astype('str')

In [3]:
#Теперь сгруппируем по мероприятиям
ebt_grouped=events_by_teacher[events_by_teacher['Имя на платформе']!='0'].groupby(['Мероприятие','Дата']). \
    agg(event_type=('Формат мероприятия','max'),
        track=('ТРЭК','max'),
        long=('Длительность','max'),
        teachers=('Имя на платформе', ','.join),
        leaderID=('leaderID',','.join)).fillna(0).reset_index()

ebt_grouped = ebt_grouped.rename(columns={'event_type':'Формат мероприятия','track':'ТРЭК','long':'Длительность',
                                         'teachers':'Препод'})

#У нас несколько треков у одного препода, так что сгруппируем по leaderID и выведем все треки по данным мероприятий
et_grouped = events_by_teacher[events_by_teacher.leaderID!='0'].groupby(['leaderID']). \
    agg(match_name=('Имя на платформе','max'),
        leaderID_list=('leaderID','max'),
        tracks=('ТРЭК', ','.join)
        ).fillna(0).reset_index()
et_grouped.leaderID = et_grouped.leaderID.astype('int')
et_grouped = et_grouped.rename(columns={ 'match_name':'Преподаватель', 'tracks':'Список трэков','leaderID_list':'leaderID'})

In [4]:
#Запишем  в файл 
with pd.ExcelWriter('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/Teacher_platform_31_12_2020.xlsx') as writer:  
    teachers_platform[teachers_platform.leaderID==0][['Преподаватель','Имя','Фамилия']] \
    .to_excel(writer, sheet_name='Не найдено соответствия',index=False)
    tp_grouped.to_excel(writer, sheet_name='Матч Препод-leaderID',index=False)   
    events_by_teacher.to_excel(writer, sheet_name='Препода и Мероприятия',index=False)
    ebt_grouped.to_excel(writer, sheet_name='Группировка по преподам',index=False)